In [4]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd 


In [5]:
data = pd.read_csv("../data/Teams.csv")
training_data = pd.read_csv("../data/Teams.csv")

In [6]:
data["yearID"] = pd.to_datetime(data["yearID"],format="%Y").dt.year
training_data['yearID'] = pd.to_datetime(training_data['yearID'],format="%Y").dt.year

In [7]:
year = 2000
data_s = data.loc[data["yearID"]>year]

In [8]:
data_s.drop(['lgID','teamID','franchID','divID','teamIDBR','teamIDlahman45','teamIDretro'],axis=1,inplace=True)
training_data.drop(['lgID','teamID','franchID','divID','teamIDBR','teamIDlahman45','teamIDretro'],axis=1,inplace=True)

In [9]:
 team = list(data_s["name"].unique())

In [10]:
data_s.columns

Index(['yearID', 'Rank', 'G', 'Ghome', 'W', 'L', 'DivWin', 'WCWin', 'LgWin',
       'WSWin', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB', 'CS',
       'HBP', 'SF', 'RA', 'ER', 'ERA', 'CG', 'SHO', 'SV', 'IPouts', 'HA',
       'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP', 'name', 'park', 'attendance',
       'BPF', 'PPF'],
      dtype='object')

In [11]:
data_s.drop(["G","Ghome","DivWin","WCWin","2B","3B","HR","SO","CS","SF","RA","CG","SHO","SV","IPouts","HRA","BBA","SOA","FP"],axis=1,inplace=True)

In [12]:
training_data.drop(["G","Ghome","DivWin","WCWin","2B","3B","HR","SO","CS","SF","RA","CG","SHO","SV","IPouts","HRA","BBA","SOA","FP"],axis=1,inplace=True)

In [13]:
len(data_s["park"].unique())

59

In [14]:
training_data.drop(["name","park"],axis=1,inplace=True)

In [15]:
training_data.head(10)

,yearID,Rank,W,L,LgWin,WSWin,R,AB,H,BB,SB,HBP,ER,ERA,HA,E,DP,attendance,BPF,PPF
0,1871,3,20,10,N,NaN,401,1372,426,60,73.0,NaN,109,3.55,367,225,NaN,NaN,103,98
1,1871,2,19,9,N,NaN,302,1196,323,60,69.0,NaN,77,2.76,308,218,NaN,NaN,104,102
2,1871,8,10,19,N,NaN,249,1186,328,26,18.0,NaN,116,4.11,346,223,NaN,NaN,96,100
3,1871,7,7,12,N,NaN,137,746,178,33,16.0,NaN,97,5.17,261,163,NaN,NaN,101,107
4,1871,5,16,17,N,NaN,302,1404,403,33,46.0,NaN,121,3.72,373,227,NaN,NaN,90,88
5,1871,1,21,7,Y,NaN,376,1281,410,46,56.0,NaN,137,4.95,329,194,NaN,NaN,102,98
6,1871,9,4,21,N,NaN,231,1036,274,38,53.0,NaN,108,4.30,315,220,NaN,NaN,97,99
7,1871,6,13,15,N,NaN,351,1248,384,49,62.0,NaN,153,5.51,431,198,NaN,NaN,101,100
8,1871,4,15,15,N,NaN,310,1353,375,48,48.0,NaN,137,4.37,371,217,NaN,NaN,94,98
9,1872,2,35,19,N,NaN,617,2576,747,27,35.0,NaN,173,3.02,566,432,NaN,NaN,106,102


In [16]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
for column in training_data:
    ax.bar(column,(training_data[column].isnull().sum()/len(training_data))*100,width=0.25)
plt.show()   

<Figure size 640x480 with 1 Axes>

In [17]:
training_data.isnull().sum()

yearID           0
Rank             0
W                0
L                0
LgWin           28
WSWin          357
R                0
AB               0
H                0
BB               0
SB             144
HBP           2325
ER               0
ERA              0
HA               0
E                0
DP             317
attendance     279
BPF              0
PPF              0
dtype: int64

In [18]:
training_data.drop("HBP",axis=1,inplace=True)
data_s.drop("HBP",axis=1,inplace=True)

In [19]:
#replacing the Nan values with 0
training_data["LgWin"] = training_data["LgWin"].fillna(0)
training_data["WSWin"] = training_data["WSWin"].fillna(0)


In [20]:
labels = training_data["Rank"].copy()

In [21]:
training_data.drop("Rank",axis=1,inplace=True)

In [22]:
training_data.head(10)

,yearID,W,L,LgWin,WSWin,R,AB,H,BB,SB,ER,ERA,HA,E,DP,attendance,BPF,PPF
0,1871,20,10,N,0,401,1372,426,60,73.0,109,3.55,367,225,NaN,NaN,103,98
1,1871,19,9,N,0,302,1196,323,60,69.0,77,2.76,308,218,NaN,NaN,104,102
2,1871,10,19,N,0,249,1186,328,26,18.0,116,4.11,346,223,NaN,NaN,96,100
3,1871,7,12,N,0,137,746,178,33,16.0,97,5.17,261,163,NaN,NaN,101,107
4,1871,16,17,N,0,302,1404,403,33,46.0,121,3.72,373,227,NaN,NaN,90,88
5,1871,21,7,Y,0,376,1281,410,46,56.0,137,4.95,329,194,NaN,NaN,102,98
6,1871,4,21,N,0,231,1036,274,38,53.0,108,4.30,315,220,NaN,NaN,97,99
7,1871,13,15,N,0,351,1248,384,49,62.0,153,5.51,431,198,NaN,NaN,101,100
8,1871,15,15,N,0,310,1353,375,48,48.0,137,4.37,371,217,NaN,NaN,94,98
9,1872,35,19,N,0,617,2576,747,27,35.0,173,3.02,566,432,NaN,NaN,106,102


In [23]:
num_attributes=["yearID","W","L","R","AB","H","BB","SB","ER","ERA","HA","E","DP","attendance","BPF","PPF"]
cat_attributes = ["LgWin","WSWin"]

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
num_pipeline =Pipeline(
    [
        ('imputer',SimpleImputer(strategy="median")),
        ('std_scaler',StandardScaler())
    ]
)

full_pipeline = ColumnTransformer([
    ("nums",num_pipeline,num_attributes),
    ("cat",OneHotEncoder(),cat_attributes)
])



In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(training_data,labels,test_size=0.2,random_state=42)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(2244, 18) (561, 18) (2244,) (561,)
